<a href="https://colab.research.google.com/github/BFvandy/EV_Buzz_Analysis/blob/main/EV_Buzz_Analysis_SW%26Reddit_Prefiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
cur_path = "/content/drive/My Drive/EV Buzz Analysis"
os.chdir(cur_path)
!pwd

/content/drive/My Drive/EV Buzz Analysis


# Load Transformer

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 13.3 MB/s 
     |████████████████████████████████| 6.6 MB 52.5 MB/s 
     |████████████████████████████████| 120 kB 51.0 MB/s 


In [ ]:
from transformers import pipeline

In [ ]:
model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name, device=0)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

In [ ]:
res

{'score': 0.2117149531841278,
 'start': 59,
 'end': 84,
 'answer': 'gives freedom to the user'}

# Load Dataset

In [ ]:
comments = pd.read_csv('EV_SW&Reddit_v2.csv', encoding_errors='ignore')
comments = comments.rename(columns={'Verbatim Text': 'Text'})
comments = comments.drop(comments.columns[[0]],axis = 1)
comments

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (4,6,8,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Make,Model,Source,Purchased,Purchased Year,Disposed,Disposed Model Year,Rejected,Rejected Model Year,Sentiment,Age Range,Gender,Ethnicity,Income Bracket,Text
0,Nissan,Leaf,Strategic Window,Nissan Leaf S Plus,2022.0,(None),NaN,Chevrolet Bolt EV,2022.0,Sentiment Not Calculated,Age 47,Male,White,"$65,001 - $70,000",It's a great commuter car. It's comfortable an...
1,Volkswagen,ID,Strategic Window,Volkswagen ID.4 AWD,2021.0,BMW X3 xDrive 30e,2018.0,Hyundai Kona,2021.0,Sentiment Not Calculated,Age 58,Male,White,"$200,001 - $300,000",LOVE THE CAR BUT...\n1) Dealer was dishonest a...
2,Tesla,Model 3,Strategic Window,Tesla Model 3,2022.0,(None),NaN,Tesla Model 3,2019.0,Sentiment Not Calculated,Age 45,Male,White,"$65,001 - $70,000",The range and energy efficiency is excellent- ...
3,Volkswagen,ID,Strategic Window,Volkswagen ID.4 AWD,2021.0,Toyota RAV4,2016.0,Volvo C40,2021.0,Sentiment Not Calculated,Age 74,Male,White,"$300,001 - $400,000",I preordered using the VW online site December...
4,Tesla,Model 3,Strategic Window,Tesla Model 3,2022.0,Nissan Maxima,1995.0,Ford Mustang Mach-E,2022.0,Sentiment Not Calculated,Age 72,Male,NaN,"$80,001 - $90,000",I just finished my trip in my 2022 Model 3 lon...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155313,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Not to jump to too many conclusions, but in ph..."
155314,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mine went from 30-1 to 30-2, so must be gettin..."
155315,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I think they're administering the update at 30...
155316,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yup. mine went 30-2 today also. i'm not far fr...


In [ ]:
fun_labels = ['Dependable', 'Lasts long', 'Value for money', 'Quality fit and finish', 'Attractive styling'
                                    , 'Safe', 'Retains resale value', 'Driver comfort', 'Fun to drive', 
                                    'Advanced features', 'Responsive handling', 'Prestigious', 'Dealerships', 
        'Fuel efficient', 'Quick acceleration', 'Environmentally friendly', 'Affordable']
pers_labels = ['Trusted', 'Leader', 'Responsible', 'Confident', 'Innovative', 'Exciting', 'Practical', 'Adventurous', 
               'Passionate', 'Distinctive', 'Youthful', 'Aggressive']

all_labels = fun_labels+pers_labels
all_labels_lower = [i.lower() for i in all_labels]

In [ ]:
comments[all_labels_lower] = pd.NaT

In [ ]:
pd.set_option('display.max_columns', None)
comments

,Make,Model,Source,Purchased,Purchased Year,Disposed,Disposed Model Year,Rejected,Rejected Model Year,Sentiment,Age Range,Gender,Ethnicity,Income Bracket,Text,dependable,lasts long,value for money,quality fit and finish,attractive styling,safe,retains resale value,driver comfort,fun to drive,advanced features,responsive handling,prestigious,dealerships,fuel efficient,quick acceleration,environmentally friendly,affordable,trusted,leader,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Nissan,Leaf,Strategic Window,Nissan Leaf S Plus,2022.0,(None),NaN,Chevrolet Bolt EV,2022.0,Sentiment Not Calculated,Age 47,Male,White,"$65,001 - $70,000",It's a great commuter car. It's comfortable an...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
1,Volkswagen,ID,Strategic Window,Volkswagen ID.4 AWD,2021.0,BMW X3 xDrive 30e,2018.0,Hyundai Kona,2021.0,Sentiment Not Calculated,Age 58,Male,White,"$200,001 - $300,000",LOVE THE CAR BUT...\n1) Dealer was dishonest a...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2,Tesla,Model 3,Strategic Window,Tesla Model 3,2022.0,(None),NaN,Tesla Model 3,2019.0,Sentiment Not Calculated,Age 45,Male,White,"$65,001 - $70,000",The range and energy efficiency is excellent- ...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
3,Volkswagen,ID,Strategic Window,Volkswagen ID.4 AWD,2021.0,Toyota RAV4,2016.0,Volvo C40,2021.0,Sentiment Not Calculated,Age 74,Male,White,"$300,001 - $400,000",I preordered using the VW online site December...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
4,Tesla,Model 3,Strategic Window,Tesla Model 3,2022.0,Nissan Maxima,1995.0,Ford Mustang Mach-E,2022.0,Sentiment Not Calculated,Age 72,Male,NaN,"$80,001 - $90,000",I just finished my trip in my 2022 Model 3 lon...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155313,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Not to jump to too many conclusions, but in ph...",NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
155314,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mine went from 30-1 to 30-2, so must be gettin...",NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
155315,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I think they're administering the update at 30...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
155316,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yup. mine went 30-2 today also. i'm not far fr...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT


# Implementation

In [ ]:
import time

start = time.time()

for i in range(len(comments)):
  for j in all_labels_lower:
    try:
      QA_input = {
          'question': 'Is ' + j + ' mentioned?',
          'context': comments.at[i, 'Text']
      }
      res = nlp(QA_input)
      comments.at[i, j] = 1 if res['score'] > 0.1 else 0 
      # i is a numeric value and j is text
    except:
      pass
  if i % 5000 == 0:
    print(i)
  else:
    pass
    
end = time.time()
print((end - start)/60)


/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1039: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000
130000
135000
140000
145000
150000
155000
1081.575152015686


In [ ]:
comments

,Make,Model,Source,Purchased,Purchased Year,Disposed,Disposed Model Year,Rejected,Rejected Model Year,Sentiment,Age Range,Gender,Ethnicity,Income Bracket,Text,dependable,lasts long,value for money,quality fit and finish,attractive styling,safe,retains resale value,driver comfort,fun to drive,advanced features,responsive handling,prestigious,dealerships,fuel efficient,quick acceleration,environmentally friendly,affordable,trusted,leader,responsible,confident,innovative,exciting,practical,adventurous,passionate,distinctive,youthful,aggressive
0,Nissan,Leaf,Strategic Window,Nissan Leaf S Plus,2022.0,(None),NaN,Chevrolet Bolt EV,2022.0,Sentiment Not Calculated,Age 47,Male,White,"$65,001 - $70,000",It's a great commuter car. It's comfortable an...,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Volkswagen,ID,Strategic Window,Volkswagen ID.4 AWD,2021.0,BMW X3 xDrive 30e,2018.0,Hyundai Kona,2021.0,Sentiment Not Calculated,Age 58,Male,White,"$200,001 - $300,000",LOVE THE CAR BUT...\n1) Dealer was dishonest a...,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Tesla,Model 3,Strategic Window,Tesla Model 3,2022.0,(None),NaN,Tesla Model 3,2019.0,Sentiment Not Calculated,Age 45,Male,White,"$65,001 - $70,000",The range and energy efficiency is excellent- ...,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Volkswagen,ID,Strategic Window,Volkswagen ID.4 AWD,2021.0,Toyota RAV4,2016.0,Volvo C40,2021.0,Sentiment Not Calculated,Age 74,Male,White,"$300,001 - $400,000",I preordered using the VW online site December...,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Tesla,Model 3,Strategic Window,Tesla Model 3,2022.0,Nissan Maxima,1995.0,Ford Mustang Mach-E,2022.0,Sentiment Not Calculated,Age 72,Male,NaN,"$80,001 - $90,000",I just finished my trip in my 2022 Model 3 lon...,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155313,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Not to jump to too many conclusions, but in ph...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
155314,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mine went from 30-1 to 30-2, so must be gettin...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
155315,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I think they're administering the update at 30...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
155316,Volkswagen,ID,Reddit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yup. mine went 30-2 today also. i'm not far fr...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
comments.to_csv('EV_SW&Reddit_prefiltered_v2.csv', index=False)
print("Save it")

Save it


In [ ]:
for i in range(len(all_labels_lower)):
  label = all_labels_lower[i]
  count = (comments[label] == 1).sum()
  print(label)
  print(count)